In [1]:
import sys
import os
sys.path.insert(0, "../../python")

import numpy as np
from beam.beams import laserpulse
from beam.beams import laserbeam
from beam.elements import plasma
from beam.elements import optic
from beam import interactions
from ionization import ionization
from ionization import adk
from lens import profile
from lens import bessel
from propagation import laser
import matplotlib.pyplot as plt

path = '/media/robert/Data_Storage/Data/Ionization/200um_0.25Density_Convergence/'

In [2]:
pulseParams = {'X' : 16e3,
               'Y' : 16e3,
               'T' : 100,
               'lam' : 0.8,
               'threads' : 4,
               'cyl' : True,
               'tau' : 35}

plasmaParams = {'X' : pulseParams['X'],
                'Y' : pulseParams['Y'],
                'Z' : 2.0e6,
                'n0': 0.25,
                'atom' : ionization.Ar,
                'name' : 'Plasma',
                'cyl' : True}

In [ ]:
# Create the desired intensity profile
lam = 0.8
ion = {'atom' : ionization.Ar,
       'tau' : pulseParams['tau'],
       'type' : 'gaussian'}
z0 = 0.75e6
zf = 100e6
dz = 0.5e6
sigmaIn = 15e4
sigmaOut = 15e4
N = 100000
z, I = profile.smoothed_gaussian_ramps(z0, dz, sigmaIn, sigmaOut, ion, N, zf)

params = {
    'Nr' : 5000,
    'R' : pulseParams['X']*.75,
    'w' : 200,
    'lam' : 0.8
}
r, E = bessel.bessel_expansion(params, z, I)

In [ ]:
tests = [{ # t direction
    'Nt' : 2**7,
    'Nx' : 2**10,
    'Ny' : 2**10,
    'Nz' : 2**8,
    'path' : path + '128X1024X1024X256/'
}, {
    'Nt' : 2**8,
    'path' : path + '256X1024X1024X256/'
}, { # x direction
    'Nt' : 2**6,
    'Nx' : 2**11,
    'Ny' : 2**10,
    'Nz' : 2**8,
    'path' : path + '64X2048X1024X256/'
}, {
    'Nx' : 2**12,
    'path' : path + '64X4096X1024X256/'
}, { # z direction
    'Nt' : 2**6,
    'Nx' : 2**10,
    'Ny' : 2**10,
    'Nz' : 2**9,
    'path' : path + '64X1024X1024X512/'
}, {
    'Nz' : 2**10,
    'path' : path + '64X1024X1024X1024/'
}, { # x-y direction
    'Nt' : 2**6,
    'Nx' : 2**11,
    'Ny' : 2**11,
    'Nz' : 2**8,
    'path' : path + '64X2048X2048X512/'
}]
for i in range(len(tests)):
    for name in tests[i]:
        pulseParams[name] = tests[i][name]
        plasmaParams[name] = tests[i][name]
    pulseParams['name'] = 'GaussianBeam'
    beam = laserbeam.Laser(pulseParams)
    pulseParams['name'] = 'GaussianPulse'
    pulse = laserpulse.Pulse(pulseParams)
    argon = plasma.UniformPlasma(plasmaParams)

    e = beam.reconstruct_from_cyl(r, E, beam.x, beam.y)
    beam.initialize_field(e)
    e = e[None, :, :] * np.exp(-pulse.t[:, None, None]**2 * np.pi/(2*pulse.tau**2))
    pulse.initialize_field(e)

    beam.propagate(np.linspace(0, plasmaParams['Z'], plasmaParams['Nz']), 1.0)
    interactions.pulse_plasma(pulse, argon)